In [23]:
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

In [24]:
from sqlalchemy import create_engine

In [25]:
sqlitedb_path = '../data/raw/mmaragon.db'
#sqlitedb_path = '/home/miguelmaragon/Documentos/IRONHACK/Ironhack-Module-1-Project---Pipelines---MMA/data/raw/mmaragon.db'

In [26]:
engine = create_engine(f'sqlite:///{sqlitedb_path}')

In [27]:
df_sqlite_b_info = pd.read_sql_query("select * from business_info", engine)
df_sqlite_p_info = pd.read_sql_query("select * from personal_info", engine)
df_sqlite_r_info = pd.read_sql_query("select * from rank_info", engine)

In [28]:
df_sqlite_b_info = df_sqlite_b_info.drop(["realTimeWorth", "Unnamed: 0"], axis=1)

In [29]:
df_sqlite_p_info = df_sqlite_p_info.drop(["Unnamed: 0"], axis=1)

In [30]:
def title_COLUMN(df, column):
    df[column] = df[column].str.title()
    return df[column]
title_COLUMN(df_sqlite_p_info, 'lastName')

0            Bezos
1            Gates
2          Buffett
3          Arnault
4       Zuckerberg
           ...    
2203          Zhao
2204          Zhou
2205           Zhu
2206          Zhuo
2207         Zorlu
Name: lastName, Length: 2208, dtype: object

In [31]:
df_sqlite_r_info = df_sqlite_r_info.drop(["Unnamed: 0"], axis=1)

In [32]:
title_COLUMN(df_sqlite_r_info, 'name')

0       Anant Asavabhokhin & Family
1                    Arash Ferdowsi
2          Banthoon Lamsam & Family
3            Bill Adderley & Family
4                 Boris Zingarevich
                   ...             
2255                 Zhao Xiaoqiang
2256                Zhou Liangzhang
2257                   Zhu Xingming
2258                       Zhuo Jun
2259                    Olgun Zorlu
Name: name, Length: 2260, dtype: object

In [33]:
df_sqlite_b_info.isnull().sum()

id                  0
Source              0
worth               0
worthChange         0
realTimePosition    0
dtype: int64

In [34]:
df_sqlite_p_info.isnull().sum()

id           0
lastName     0
age         65
gender      13
country      0
image        0
dtype: int64

In [35]:
df_sqlite_r_info.isnull().sum()

id           0
name         0
position    52
dtype: int64

In [36]:
from functools import reduce
dfs = [df_sqlite_b_info, df_sqlite_p_info, df_sqlite_r_info]
df_final = reduce(lambda left,right: pd.merge(left,right,on='id'), dfs)

In [37]:
df_final = df_final[['id', 'position', 'name', 'lastName', 'age','gender','country','Source','worth', 'image']]
df_final.rename(columns = {'id':'Id','position':'Position','name':'Name','lastName':'LastName','age':'Age','gender':'Gender','country':'Country','worth':'Worth','image':'Image'}, inplace = True)

In [38]:
df_final.Position = df_final.Position.astype(int)
df_final = df_final.sort_values(by='Position', ascending=True)

In [39]:
df_final

,Id,Position,Name,LastName,Age,Gender,Country,Source,Worth,Image
0,5110,1,Jeff Bezos,Bezos,54 years old,M,United States,Technology ==> Amazon,112.0 BUSD,https://specials-images.forbesimg.com/imageser...
1,8029,2,Bill Gates,Gates,62 years old,None,United States,Technology ==> Microsoft,90.0 BUSD,https://specials-images.forbesimg.com/imageser...
2,1017,3,Warren Buffett,Buffett,87 years old,None,United States,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,https://specials-images.forbesimg.com/imageser...
3,3437,4,Bernard Arnault,Arnault,69 years old,None,None,Fashion & Retail ==> LVMH,72.0 BUSD,https://specials-images.forbesimg.com/imageser...
4,4589,5,Mark Zuckerberg,Zuckerberg,1985,None,United States,Technology ==> Facebook,71.0 BUSD,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...,...,...,...,...,...
2203,1536,2204,Zhao Xiaoqiang,Zhao,51 years old,M,China,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,https://specials-images.forbesimg.com/imageser...
2204,3722,2205,Zhou Liangzhang,Zhou,55 years old,M,None,Manufacturing ==> electrical equipment,1.0 BUSD,https://specials-images.forbesimg.com/imageser...
2205,7308,2206,Zhu Xingming,Zhu,51 years old,M,China,Manufacturing ==> electrical equipment,1.0 BUSD,https://specials-images.forbesimg.com/imageser...
2206,7816,2207,Zhuo Jun,Zhuo,52 years old,None,Hong Kong,Manufacturing ==> printed circuit boards,1.0 BUSD,https://specials-images.forbesimg.com/imageser...


In [18]:
def Gender_f(the_row):
    if the_row == "M":
        return "Male"
    elif the_row == "F":
        return "Female"
    else:
        return the_row
def Age_f(the_row):
    if the_row >= 1000:
        return now.year - the_row
    else:
        return the_row

In [19]:
df_final['Gender'] = df_final.apply(lambda row: Gender_f(row.Gender), axis=1)

df_final['Age'] = df_final.Age.str.replace(' years old', '')
df_final['Age'] = df_final.Age.fillna(0)
df_final['Age'] = df_final.Age.astype(int)
df_final['Age'] = df_final.apply(lambda row: Age_f(row.Age), axis=1)


df_final['Worth'] = df_final.Worth.str.replace(' BUSD', '')
df_final['Worth'] = df_final.Worth.astype(float)

#df_final['WorthChange'] = df_final.WorthChange.str.replace(' millions USD', '')
#df_final['WorthChange'] = df_final.WorthChange.astype(float)

df_final['Sector'] = df_final.Source.str.split(' ==> ').str[0].str.title()
df_final['Company'] = df_final.Source.str.split(' ==> ').str[1].str.title()
df_final = df_final.drop(["Source"], axis=1)

df_final['Image'] = df_final.Image.str.replace('https:http://', 'https://')
df_final['Image'] = df_final.Image.str.replace('/n', '')

In [20]:
df_final.tail(50)

,Id,Position,Name,LastName,Age,Gender,Country,Worth,Image,Sector,Company
2158,6055,2159,Koo Bon-Sik,Koo,59,None,None,1.0,https://specials-images.forbesimg.com/imageser...,Technology,Electronics/Household Products
2159,1199,2160,Igor Kudryashkin,Kudryashkin,56,None,None,1.0,https://specials-images.forbesimg.com/imageser...,Metals & Mining,Metallurgy
2160,6851,2161,Lev Kvetnoi,Kvetnoi,52,Male,None,1.0,https://specials-images.forbesimg.com/imageser...,Finance And Investments,Cement
2161,4268,2162,Theodore Leonsis,Leonsis,62,None,None,1.0,https://specials-images.forbesimg.com/imageser...,Sports,Sports Teams
2162,7011,2163,Lev Leviev,Leviev,62,None,None,1.0,https://specials-images.forbesimg.com/imageser...,Metals & Mining,Diamonds
2163,8484,2164,Liang Zhaoxian,Liang,53,Male,China,1.0,https://specials-images.forbesimg.com/imageser...,Manufacturing,Appliances
2164,7781,2165,John Liew,Liew,50,Male,United States,1.0,https://specials-images.forbesimg.com/imageser...,Finance And Investments,Investment Management
2165,7399,2166,Paul Little,Little,70,Male,None,1.0,https://specials-images.forbesimg.com/imageser...,Real Estate,"Logistics, Real Estate"
2166,1301,2167,Liu Fuhai,Liu,64,Male,None,1.0,https://specials-images.forbesimg.com/imageser...,Diversified,"Textiles, Energy"
2167,4666,2168,Liu Jincheng,Liu,53,Male,None,1.0,https://specials-images.forbesimg.com/imageser...,Technology,Lithium Batteries


In [21]:
df_final.to_csv('../data/processed/ForbesBillionaires2018.csv', index=False)

In [22]:
df_final[['Id','Age']].groupby(['Age']).count().sort_values(by='Age', ascending=True)

,Id
Age,
0,65
21,1
23,1
25,1
27,1
...,...
94,7
95,4
96,1
